In `5_dataframe_basics` file, we noticed that that schema information is not correct in DataFrame.  
So, we will fix the datatypes.

I got CSV file from kaggle: https://www.kaggle.com/datasets/anandshaw2001/netflix-movies-and-tv-shows?resource=download

In [29]:
from pyspark.sql import *
from configparser import ConfigParser

In [30]:
config = ConfigParser()
config.read("config.ini")

dataset_path = config["file_path"]["data_set_path"]

In [31]:
spark = SparkSession.builder \
                .getOrCreate()

In [32]:
raw_df = spark.read \
            .option("inferSchema", "true") \
            .option("header", "true") \
            .csv(dataset_path)

raw_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                NULL|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

Let's know the column type

In [33]:
raw_df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



Everything is String, so we will fix the datatypes, and this is the part of the transformation.  
Before transforming, we need to know how to select one column  

- In PySpark, you cannot directly display a single column using display(raw_df["date_added"]).  
- This will result in an error because PySpark DataFrame columns are not like Pandas Series.  
- So, we need to use `select` method on that DataFrame.

Below is how we can select one column:


In [34]:
raw_df.select("date_added").show()

+------------------+
|        date_added|
+------------------+
|September 25, 2021|
|September 24, 2021|
|September 24, 2021|
|September 24, 2021|
|September 24, 2021|
|September 24, 2021|
|September 24, 2021|
|September 24, 2021|
|September 24, 2021|
|September 24, 2021|
|September 24, 2021|
|September 23, 2021|
|September 23, 2021|
|September 22, 2021|
|September 22, 2021|
|September 22, 2021|
|September 22, 2021|
|September 22, 2021|
|September 22, 2021|
|September 22, 2021|
+------------------+
only showing top 20 rows



Cool, now we will try to fix the data type of this column.  
To change column type we have function: `.withColumn()`  

its syntax: `spark_df.withColumn("old column name", "logic to tranform")`

BTW if we this data:
```
+------------------+
|        date_added|
+------------------+
|September 25, 2021|
|September 24, 2021|
```

Then its Datatype should be `str` and not other datatype.  
But on this data we cannot filter out the records based on date filter, so we need to convert it into date.  

In [35]:
from pyspark.sql.functions import * 

In [36]:
fixed_df = raw_df.withColumn("date_added", to_date("date_added", "MMMM d, yyyy"))

fixed_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|2021-09-25|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                NULL|Ama Qamata, Khosi...|        South Africa|2021-09-24|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, T

we have use `to_date`  
So, its syntax is like this: `to_date(col, format)`  

- col: The column containing the date as a string.
- format: The expected format of the input string.

#### Example with Different Formats

1. Case 1: "dd-MM-yyyy" (Day-Month-Year)
    ```
    df = df.withColumn("date_col", to_date("date_col", "dd-MM-yyyy"))
    ```

    📌 Input: "25-09-2021"  
    📌 Output: 2021-09-25 (DateType)

2. Case 2: "MMMM d, yyyy" (Full Month Name, Day, Year)
    ```
    df = df.withColumn("date_col", to_date("date_col", "MMMM d, yyyy"))
    ```
    📌 Input: "September 25, 2021"  
    📌 Output: 2021-09-25

3. Case 3: "yyyy/MM/dd" (Year/Month/Day)
    ```
    df = df.withColumn("date_col", to_date("date_col", "yyyy/MM/dd"))
    ```
    📌 Input: "2021/09/25"  
    📌 Output: 2021-09-25  

#### Important Note:  
- to_date() only extracts the date part:

-  If your column has a timestamp (e.g., "2024-02-21 14:30:00"), to_date() will remove the time.
   ```
   df = df.withColumn("date_col", to_date("date_col", "yyyy-MM-dd HH:mm:ss"))
   ```
   📌 Input: "2024-02-21 14:30:00"  
   📌 Output: 2024-02-21 (time removed)

Incorrect Format Returns **NULL**

Ok, now let's convert `release_year` column into integer.  
to convert a string column (STRINGTYPE) to an integer (INTTYPE), you can use the cast() function.

In [37]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

In [38]:
transformed_df = fixed_df.withColumn("release_year", col("release_year").cast(IntegerType()))

transformed_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|2021-09-25|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                NULL|Ama Qamata, Khosi...|        South Africa|2021-09-24|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, T

In [39]:
## let's check the schema info now

transformed_df.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)

